In [65]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [66]:
base = pd.read_csv("bases/bases_limpas/base_para_modelo_final.csv")

In [67]:
base.columns.tolist()

['Numero nota fiscal',
 'Data da venda',
 'Código produto',
 'Descrição do produto',
 'Quantidade do produto',
 'Valor unitário',
 'Preço de custo',
 'Valor total produto',
 'Valor da nota',
 'Código da categoria',
 'Categoria',
 'Código da Marca',
 'Marca',
 'Quantidade estoque',
 'Faixa de Preço',
 'Margem bruta',
 'Margem %',
 'Markup',
 'Ticket_medio']

In [68]:
def recomendar_substitutos(df, cod_produto_pesquisado, n_recomendacoes=5):
    # 1. Localiza o produto base
    produto_base = df[df['Código produto'] == cod_produto_pesquisado]
    
    if produto_base.empty:
        return pd.DataFrame({"Mensagem": ["Produto não encontrado"]})  # Retorna DataFrame vazio
    
    produto_base = produto_base.iloc[0]
    
    # 2. Verifica estoque - CORREÇÃO: comparando com 0
    if produto_base['Quantidade estoque'] > 0:
        return pd.DataFrame({"Mensagem": ["Produto em estoque"]})
    
    # 3. Filtra produtos da mesma categoria e em estoque
    candidatos = df[
        (df['Código da categoria'] == produto_base['Código da categoria']) &
        (df['Código produto'] != cod_produto_pesquisado) &
        (df['Quantidade estoque'] > 0)
    ].copy()
    
    if candidatos.empty:
        return pd.DataFrame({"Mensagem": ["Sem produtos semelhantes em estoque"]})
    
    # 4. Remove duplicatas mantendo o produto com maior margem
    candidatos = candidatos.sort_values('Margem %', ascending=False).drop_duplicates('Código produto')
    
    # 5. Cálculo de similaridade simplificado e robusto
    try:
        candidatos['dist_preco'] = abs(candidatos['Valor unitário'] - produto_base['Valor unitário'])
        candidatos['dist_margem'] = abs(candidatos['Margem %'] - produto_base['Margem %'])
        
        # Normalização
        max_preco = candidatos['dist_preco'].max() or 1  # Evita divisão por zero
        max_margem = candidatos['dist_margem'].max() or 1
        
        candidatos['similaridade'] = (
            1 - 0.7*(candidatos['dist_preco']/max_preco) + 
            0.3*(1 - candidatos['dist_margem']/max_margem)
        )
        
        # 6. Ordena e seleciona os mais similares
        recomendados = candidatos.sort_values('similaridade', ascending=False).head(n_recomendacoes)
        
        return recomendados[[
            'Código produto', 'Descrição do produto', 'Valor unitário',
            'Margem %', 'Marca', 'Quantidade estoque', 'similaridade'
        ]]
    
    except Exception as e:
        return pd.DataFrame({"Erro": [f"Falha no cálculo: {str(e)}"]})

# Chamada CORRETA da função:
# recomendacoes = recomendar_substitutos(base, 36937)  # Primeiro o dataframe, depois o código

In [71]:
# Exemplo de uso
recomendacoes = recomendar_substitutos(base, 36937)
display(recomendacoes)

,Código produto,Descrição do produto,Valor unitário,Margem %,Marca,Quantidade estoque,similaridade
87440,36945,CABO TYPE C BR CB901 OEX,15.69,0.21,CSL IMPORTADORA LTDA,15,1.242264
8460,36944,CABO TYPE C LIGHTNING BR CB900 OEX,15.69,0.21,CSL IMPORTADORA LTDA,10,1.242264
121020,36953,CARREGADOR USB BR CG202 OEX,33.73,0.21,CSL IMPORTADORA LTDA,5,1.167547
121070,36930,MOUSE PAD SHOT PT MP302 OEX,30.96,0.30,CSL IMPORTADORA LTDA,12,1.097201
113790,36957,CONVERSOR TOMADA UNIV BR CP100 OEX,29.56,0.31,CSL IMPORTADORA LTDA,5,1.093909


In [72]:
def recomendar_substitutos(df, cod_produto_pesquisado, n_recomendacoes=5):
    # 1. Localiza o produto base
    produto_base = df[df['Código produto'] == cod_produto_pesquisado]

    if produto_base.empty:
        return "Produto não encontrado"

    produto_base = produto_base.iloc[0]

    # 2. Verifica se o produto está sem estoque
    if produto_base['Quantidade estoque'] > 0:
        return "Produto em estoque, recomendação não necessária"

    # 3. Filtra produtos da mesma categoria, com estoque e código diferente
    candidatos = df[
        (df['Código da categoria'] == produto_base['Código da categoria']) &
        (df['Código produto'] != cod_produto_pesquisado) &
        (df['Quantidade estoque'] > 0)
    ].copy()

    if candidatos.empty:
        return "Sem produtos semelhantes em estoque"

    # 4. Remove duplicatas por código de produto
    candidatos = candidatos.drop_duplicates(subset='Código produto')

    # 5. Calcula distância de preço e margem %
    candidatos['dist_preco'] = (candidatos['Valor unitário'] - produto_base['Valor unitário']).abs()
    candidatos['dist_margem'] = (candidatos['Margem %'] - produto_base['Margem %']).abs()

    # 6. Ordena pelos mais similares
    recomendados = candidatos.sort_values(
        by=['dist_preco', 'dist_margem']
    ).head(n_recomendacoes)

    return recomendados[['Código produto', 'Descrição do produto', 'Valor unitário', 'Margem %', 'Marca', 'Quantidade estoque']]


In [82]:
recomendar_substitutos(base, 35494)

,Código produto,Descrição do produto,Valor unitário,Margem %,Marca,Quantidade estoque
24702,35485,TNT TECIDO 50M 37G AZ CE DECOFIX,54.14,0.14,JC DECOR PLASTICOS E TECIDOS LTDA,4
2546,29097,TNT TECIDO 50M 35G PK NEWTNT,54.78,0.12,NEWTNT INDUSTRIA DE TECIDOS LTDA,33
76110,14797,TNT TECIDO 50M RS SUL BRASIL,55.81,0.14,SUL BRASIL IND E COM DE ACESSORIOS DE PLAST E ...,2
76111,14840,TNT TECIDO 50M AZ RY SUL BRASIL,55.81,0.14,SUL BRASIL IND E COM DE ACESSORIOS DE PLAST E ...,4
38858,35495,TNT TECIDO 50M 37G RX DECOFIX,55.82,0.17,JC DECOR PLASTICOS E TECIDOS LTDA,30
